In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
import time
import re

In [2]:
df=pd.read_csv(r"D:\Lead Walnut\Data\internal_all.csv")
df.head()

,Address,Content Type,Status Code,Status,Indexability,Indexability Status,Title 1,Title 1 Length,Title 1 Pixel Width,Meta Description 1,...,Response Time,Last Modified,Redirect URL,Redirect Type,Cookies,Language,HTTP Version,Mobile Alternate Link,URL Encoded Address,Crawl Timestamp
0,https://www.splashtop.com/,text/html; charset=UTF-8,200,OK,Indexable,NaN,Splashtop: Secure Remote Access & Remote Suppo...,57,552,"Fast and secure solutions for remote work, rem...",...,0.896,NaN,NaN,NaN,NaN,en,1.1,NaN,https://www.splashtop.com/,12-10-2024 19:25
1,https://www.splashtop.com/resources?type=case-...,text/html; charset=UTF-8,200,OK,Non-Indexable,Canonicalised,"Splashtop Resource Library | Whitepapers, eBoo...",57,531,Stay on top of the remote access and IT suppor...,...,0.899,NaN,NaN,NaN,NaN,en,1.1,NaN,https://www.splashtop.com/resources?type=case-...,12-10-2024 19:25
2,https://www.splashtop.com/about/leadership,text/html; charset=UTF-8,200,OK,Indexable,NaN,Leadership Team & Board of Directors | Splasht...,53,478,The Splashtop leadership team is dedicated to ...,...,0.985,NaN,NaN,NaN,NaN,en,1.1,NaN,https://www.splashtop.com/about/leadership,12-10-2024 19:25
3,https://www.splashtop.com/customers,text/html; charset=UTF-8,200,OK,Indexable,NaN,Splashtop Customers - Using Splashtop Remote A...,60,578,Top businesses and educational institutions li...,...,0.906,NaN,NaN,NaN,NaN,en,1.1,NaN,https://www.splashtop.com/customers,12-10-2024 19:25
4,https://www.splashtop.com/partners/integrations,text/html; charset=UTF-8,200,OK,Indexable,NaN,Splashtop Remote Access & Support Integrations,46,435,Integrate Splashtop’s powerful solutions into ...,...,0.997,NaN,NaN,NaN,NaN,en,1.1,NaN,https://www.splashtop.com/partners/integrations,12-10-2024 19:25


In [3]:
df.shape

(205, 68)

In [4]:
df.columns

Index(['Address', 'Content Type', 'Status Code', 'Status', 'Indexability',
       'Indexability Status', 'Title 1', 'Title 1 Length',
       'Title 1 Pixel Width', 'Meta Description 1',
       'Meta Description 1 Length', 'Meta Description 1 Pixel Width',
       'Meta Keywords 1', 'Meta Keywords 1 Length', 'H1-1', 'H1-1 Length',
       'H2-1', 'H2-1 Length', 'H2-2', 'H2-2 Length', 'Meta Robots 1',
       'X-Robots-Tag 1', 'Meta Refresh 1', 'Canonical Link Element 1',
       'rel="next" 1', 'rel="prev" 1', 'HTTP rel="next" 1',
       'HTTP rel="prev" 1', 'amphtml Link Element', 'Size (bytes)',
       'Transferred (bytes)', 'Total Transferred (bytes)', 'CO2 (mg)',
       'Carbon Rating', 'Word Count', 'Sentence Count',
       'Average Words Per Sentence', 'Flesch Reading Ease Score',
       'Readability', 'Text Ratio', 'Crawl Depth', 'Folder Depth',
       'Link Score', 'Inlinks', 'Unique Inlinks', 'Unique JS Inlinks',
       '% of Total', 'Outlinks', 'Unique Outlinks', 'Unique JS Outlin

In [5]:
relevant_columns = [
    'Address',                # URL of the page
    'Content Type',           # Type of content (HTML, etc.)
    'Status Code',            # HTTP status code (e.g., 200, 301)
    'Indexability',           # Whether the page is indexable by search engines
    'Title 1',                # SEO title of the page
    'Title 1 Length',         # Length of the SEO title
    'Meta Description 1',     # Meta description of the page
    'Meta Description 1 Length', # Length of the meta description
    'H1-1',                   # Primary H1 tag
    'H2-1',                   # First H2 tag
    'H2-2',                   # Second H2 tag (if applicable)
    'Word Count',             # Total word count of the page
    'Readability',            # Readability score
    'Text Ratio',             # Ratio of text to code on the page
    'Crawl Depth',            # Crawl depth of the page in the website hierarchy
    'Inlinks',                # Number of internal links
    'Outlinks',               # Number of outbound links
    'Response Time'           # Page load time
]

df = df[relevant_columns]
df.head()


,Address,Content Type,Status Code,Indexability,Title 1,Title 1 Length,Meta Description 1,Meta Description 1 Length,H1-1,H2-1,H2-2,Word Count,Readability,Text Ratio,Crawl Depth,Inlinks,Outlinks,Response Time
0,https://www.splashtop.com/,text/html; charset=UTF-8,200,Indexable,Splashtop: Secure Remote Access & Remote Suppo...,57,"Fast and secure solutions for remote work, rem...",152,The #1 Rated Remote Access and Support Software,Trusted by 30 Million Users and 85% of Fortune...,Security and Privacy,600,Hard,1.611,0,259,104,0.896
1,https://www.splashtop.com/resources?type=case-...,text/html; charset=UTF-8,200,Non-Indexable,"Splashtop Resource Library | Whitepapers, eBoo...",57,Stay on top of the remote access and IT suppor...,130,Splashtop Resource Library,Filter Resources,NaN,270,Fairly Hard,1.629,1,221,100,0.899
2,https://www.splashtop.com/about/leadership,text/html; charset=UTF-8,200,Indexable,Leadership Team & Board of Directors | Splasht...,53,The Splashtop leadership team is dedicated to ...,134,Leadership,Executive Team,Board Members,277,Normal,1.723,1,376,89,0.985
3,https://www.splashtop.com/customers,text/html; charset=UTF-8,200,Indexable,Splashtop Customers - Using Splashtop Remote A...,60,Top businesses and educational institutions li...,160,The Best Customers are at Splashtop,Discover How We Have Helped Our Customers Thro...,3D CAD / CAM / BIM,193,Fairly Easy,0.880,1,196,92,0.906
4,https://www.splashtop.com/partners/integrations,text/html; charset=UTF-8,200,Indexable,Splashtop Remote Access & Support Integrations,46,Integrate Splashtop’s powerful solutions into ...,145,Our Integration Partners,RMM Partners,Integrate Splashtop With Your Applications Thr...,978,Hard,2.415,1,199,121,0.997


In [6]:
df_filtered = df[df['Content Type'] != 'image/png']
df=df_filtered[df_filtered['Indexability']!='Non-Indexable']
df.head()

,Address,Content Type,Status Code,Indexability,Title 1,Title 1 Length,Meta Description 1,Meta Description 1 Length,H1-1,H2-1,H2-2,Word Count,Readability,Text Ratio,Crawl Depth,Inlinks,Outlinks,Response Time
0,https://www.splashtop.com/,text/html; charset=UTF-8,200,Indexable,Splashtop: Secure Remote Access & Remote Suppo...,57,"Fast and secure solutions for remote work, rem...",152,The #1 Rated Remote Access and Support Software,Trusted by 30 Million Users and 85% of Fortune...,Security and Privacy,600,Hard,1.611,0,259,104,0.896
2,https://www.splashtop.com/about/leadership,text/html; charset=UTF-8,200,Indexable,Leadership Team & Board of Directors | Splasht...,53,The Splashtop leadership team is dedicated to ...,134,Leadership,Executive Team,Board Members,277,Normal,1.723,1,376,89,0.985
3,https://www.splashtop.com/customers,text/html; charset=UTF-8,200,Indexable,Splashtop Customers - Using Splashtop Remote A...,60,Top businesses and educational institutions li...,160,The Best Customers are at Splashtop,Discover How We Have Helped Our Customers Thro...,3D CAD / CAM / BIM,193,Fairly Easy,0.880,1,196,92,0.906
4,https://www.splashtop.com/partners/integrations,text/html; charset=UTF-8,200,Indexable,Splashtop Remote Access & Support Integrations,46,Integrate Splashtop’s powerful solutions into ...,145,Our Integration Partners,RMM Partners,Integrate Splashtop With Your Applications Thr...,978,Hard,2.415,1,199,121,0.997
5,https://www.splashtop.com/solutions/iot,text/html; charset=UTF-8,200,Indexable,Remote Access for IoT Devices: Manage & Suppor...,61,Manage and support IoT devices with Splashtop ...,139,Remote Access for IoT Devices - Manage & Suppo...,Remote Support for IoT Devices: Streamline Sup...,Industry Solutions,1154,Hard,4.067,1,200,94,1.014


In [7]:
df.shape

(172, 18)

In [6]:
# df["Address"].to_csv("links.csv")
df2=pd.read_csv(r"D:\Lead Walnut\links.csv")
df2

,URL_ID,URL
0,0,https://www.splashtop.com/
1,2,https://www.splashtop.com/about/leadership
2,3,https://www.splashtop.com/customers
3,4,https://www.splashtop.com/partners/integrations
4,5,https://www.splashtop.com/solutions/iot
...,...,...
167,198,https://www.splashtop.com/compare/logmein-resc...
168,199,https://www.splashtop.com/compare/logmein-alte...
169,200,https://www.splashtop.com/compare/teamviewer-a...
170,201,https://www.splashtop.com/compare/beyondtrust-...


# Anydesk

In [18]:
df=pd.read_csv(r"D:\Lead Walnut\Data\internal_all_anydesk.csv")
df.head()

,Address,Content Type,Status Code,Status,Indexability,Indexability Status,Title 1,Title 1 Length,Title 1 Pixel Width,Meta Description 1,...,Response Time,Last Modified,Redirect URL,Redirect Type,Cookies,Language,HTTP Version,Mobile Alternate Link,URL Encoded Address,Crawl Timestamp
0,https://anydesk.com/en,text/html;charset=UTF-8,200,OK,Indexable,NaN,The Fast Remote Desktop Application – AnyDesk,45,433,"Discover AnyDesk, the secure & intuitive remot...",...,0.042,NaN,NaN,NaN,NaN,en,1.1,NaN,https://anydesk.com/en,2024-10-13 12:06:24
1,https://anydesk.com/_static/img/icons/twitter-...,image/svg+xml;charset=UTF-8,200,OK,Indexable,NaN,NaN,0,0,NaN,...,0.033,NaN,NaN,NaN,NaN,NaN,1.1,NaN,https://anydesk.com/_static/img/icons/twitter-...,2024-10-13 12:06:27
2,https://anydesk.com/_static/video-js/video-js....,text/css;charset=UTF-8,200,OK,Indexable,NaN,NaN,0,0,NaN,...,0.044,NaN,NaN,NaN,NaN,NaN,1.1,NaN,https://anydesk.com/_static/video-js/video-js....,2024-10-13 12:06:27
3,https://anydesk.com/_static/img/sliders/refere...,application/octet-stream;charset=UTF-8,200,OK,Indexable,NaN,NaN,0,0,NaN,...,0.034,NaN,NaN,NaN,NaN,NaN,1.1,NaN,https://anydesk.com/_static/img/sliders/refere...,2024-10-13 12:06:27
4,https://anydesk.com/_static/icons/white/bold/a...,image/svg+xml;charset=UTF-8,200,OK,Indexable,NaN,NaN,0,0,NaN,...,0.035,NaN,NaN,NaN,NaN,NaN,1.1,NaN,https://anydesk.com/_static/icons/white/bold/a...,2024-10-13 12:06:27


In [19]:
# Select only the relevant columns for content analysis
relevant_columns = [
    'Address',                # URL of the page
    'Content Type',           # Type of content (HTML, etc.)
    'Status Code',            # HTTP status code (e.g., 200, 301)
    'Indexability',           # Whether the page is indexable by search engines
    'Title 1',                # SEO title of the page
    'Title 1 Length',         # Length of the SEO title
    'Meta Description 1',     # Meta description of the page
    'Meta Description 1 Length', # Length of the meta description
    'H1-1',                   # Primary H1 tag
    'H2-1',                   # First H2 tag
    'H2-2',                   # Second H2 tag (if applicable)
    'Word Count',             # Total word count of the page
    'Readability',            # Readability score
    'Text Ratio',             # Ratio of text to code on the page
    'Crawl Depth',            # Crawl depth of the page in the website hierarchy
    'Inlinks',                # Number of internal links
    'Outlinks',               # Number of outbound links
    'Response Time'           # Page load time
]

df = df[relevant_columns]
df.head()


,Address,Content Type,Status Code,Indexability,Title 1,Title 1 Length,Meta Description 1,Meta Description 1 Length,H1-1,H2-1,H2-2,Word Count,Readability,Text Ratio,Crawl Depth,Inlinks,Outlinks,Response Time
0,https://anydesk.com/en,text/html;charset=UTF-8,200,Indexable,The Fast Remote Desktop Application – AnyDesk,45,"Discover AnyDesk, the secure & intuitive remot...",126,NaN,See what's new,"Trusted by over 200,000 customers",602,Hard,5.913,0,108,141,0.042
1,https://anydesk.com/_static/img/icons/twitter-...,image/svg+xml;charset=UTF-8,200,Indexable,NaN,0,NaN,0,NaN,NaN,NaN,0,NaN,0.000,1,0,0,0.033
2,https://anydesk.com/_static/video-js/video-js....,text/css;charset=UTF-8,200,Indexable,NaN,0,NaN,0,NaN,NaN,NaN,0,NaN,0.000,1,0,0,0.044
3,https://anydesk.com/_static/img/sliders/refere...,application/octet-stream;charset=UTF-8,200,Indexable,NaN,0,NaN,0,NaN,NaN,NaN,0,NaN,0.000,1,0,0,0.034
4,https://anydesk.com/_static/icons/white/bold/a...,image/svg+xml;charset=UTF-8,200,Indexable,NaN,0,NaN,0,NaN,NaN,NaN,0,NaN,0.000,1,0,0,0.035


In [20]:
df.shape

(422, 18)

In [21]:
df_filtered = df[df['Content Type'] == 'text/html;charset=UTF-8']
df_filtered.shape

(114, 18)

In [24]:
df=df_filtered[df_filtered['Indexability']!='Non-Indexable']
print(df.shape)
df.head()

(109, 18)


,Address,Content Type,Status Code,Indexability,Title 1,Title 1 Length,Meta Description 1,Meta Description 1 Length,H1-1,H2-1,H2-2,Word Count,Readability,Text Ratio,Crawl Depth,Inlinks,Outlinks,Response Time
0,https://anydesk.com/en,text/html;charset=UTF-8,200,Indexable,The Fast Remote Desktop Application – AnyDesk,45,"Discover AnyDesk, the secure & intuitive remot...",126,NaN,See what's new,"Trusted by over 200,000 customers",602,Hard,5.913,0,108,141,0.042
9,https://anydesk.com/en/case-studies/south-boun...,text/html;charset=UTF-8,200,Indexable,AnyDesk for Remote Support – Case Study Southb...,56,Find out about Southbound Brook Public School’...,143,Intuitive Remote Support Software for Educatio...,Working fast in a secure environment,NaN,892,Hard,8.604,1,32,139,2.201
10,https://anydesk.com/en/case-studies/photograph...,text/html;charset=UTF-8,200,Indexable,AnyDesk for Remote Work – Case Study Deniz Saylan,49,Discover how self-employed photographer Deniz ...,160,Freelance Photographer Manages Large Amounts o...,Easy management of large amounts of data,NaN,855,Fairly Hard,7.994,1,30,139,2.198
11,https://anydesk.com/en/case-studies/sunchemical,text/html;charset=UTF-8,200,Indexable,AnyDesk for Remote Support – Case Study Sun Ch...,52,International enterprise uses AnyDesk On-Premi...,121,International Ultimate Uses AnyDesk On-Premise...,Connecting a decentralized global IT team with...,NaN,850,Hard,8.263,1,34,139,2.341
12,https://anydesk.com/en/performance,text/html;charset=UTF-8,200,Indexable,AnyDesk: The High-Performance Remote Access Tool,48,AnyDesk’s low latency is imperceptible to the ...,109,How Outstanding Performance Saves You Valuable...,"Discover industry-leading performance, high fr...",Why AnyDesk is so fast,605,Fairly Hard,6.150,1,307,130,2.394


In [28]:
# df["Address"].to_csv("links1.csv")
df2 = pd.read_csv(r"D:\Lead Walnut\links1.csv")
df2 = df2.drop('Unnamed: 0', axis=1)
df2['URL_ID'] = df2.index + 1

In [37]:
df2 = pd.read_csv(r"D:\Lead Walnut\links1.csv")
df2 = df2.drop('Unnamed: 0', axis=1)
df2 = df2.iloc[:, 0:1]  # Select only the first column
df2.columns = ['URL']  # Rename the column to 'URL'
df2['URL_ID'] = df2.index + 1
df2 = df2[['URL_ID', 'URL']]  # Reorder columns

In [36]:
df2.to_csv("links3.csv")

In [40]:
df2

,URL_ID,URL
0,1,https://anydesk.com/en
1,2,https://anydesk.com/en/case-studies/south-boun...
2,3,https://anydesk.com/en/case-studies/photograph...
3,4,https://anydesk.com/en/case-studies/sunchemical
4,5,https://anydesk.com/en/performance
...,...,...
104,105,https://anydesk.com/en/changelog/windows
105,106,https://anydesk.com/en/downloads/freebsd
106,107,https://anydesk.com/en/resources/education-guide
107,108,https://anydesk.com/en/case-studies/autoform-b...
